In [9]:
import requests

def download_wikipedia(dump_url, choice, 
                       dump_file_path='wikipedia_dump_file.bz2'):
    """
    Downloads the full Wikipedia dump or a 5 MB subset.

    Input Parameters:
    - dump_url (str): The URL of the Wikipedia dump file to be downloaded.
    - choice (int): Enter 0 for the full download or 1 for a 5 MB subset.
    - dump_file_path (str, optional): Path where the downloaded file will be
      saved (default: 'wikipedia_dump_file.bz2').
    
    Returns:
    - str: The path to the saved dump file.
    """

    # Stream the file download based on user's choice (0 = full, 1 = subset)
    with requests.get(dump_url, stream=True) as r, open(dump_file_path, 'wb') as f:
        r.raise_for_status()  # Raise an error for bad responses
        
        if choice == 1:  # Download 5 MB subset
            downloaded_size = 0
            for chunk in r.iter_content(1024):  # Download data in chunks of 1 KB
                if chunk:
                    f.write(chunk)  # Write the chunk to the file
                    downloaded_size += len(chunk)  # Increase size counter
                    if downloaded_size >= 5_000_000:  # Stop after 5 MB
                        break
        elif choice == 0:  # Download the full dump
            for chunk in r.iter_content(1024):
                if chunk:
                    f.write(chunk)
            print(f"Downloaded full dump.")
    
    print(f"Dump saved to {dump_file_path}")
    return dump_file_path

user_choice = int(input("Enter 1 to download a 5MB subset or 0 for the full dump: "))
dump_url = 'https://dumps.wikimedia.org/enwiki/latest/enwiki-latest-pages-articles.xml.bz2'
dump = download_wikipedia(dump_url, user_choice)

Dump saved to wikipedia_dump_file.bz2


In [ ]:
import os
import subprocess

def extract_wikipedia_dump(dump_file_path, output_dir='wikipedia_extracted', is_subset=True):
    """
    Extracts plain text from the Wikipedia dump using WikiExtractor.

    Parameters:
    dump_file_path (str): The path to the downloaded Wikipedia dump file.
    output_dir (str, optional): The directory where the extracted text will be saved.
                                Defaults to 'wikipedia_extracted'.
    is_subset (bool, optional): If True, assume the dump file is a small subset (e.g., 5MB).
                                If False, process the full dump. Defaults to True.
    """
    #  Create output directory if it does not exist
    os.makedirs(output_dir, exist_ok=True)

    #  Use '--bytes' for smaller subsets
    extractor_command = ['wikiextractor', '--json', '--no-templates', '-o', output_dir, dump_file_path]

    if is_subset:
        extractor_command.insert(1, '--bytes')
        extractor_command.insert(2, '5M')  #5  MB

    #  Run wikiextractor via subprocess
    subprocess.run(extractor_command, check=True)

    print(f"Extraction completed.  Extracted files are saved in {output_dir}")

#Test
extract_wikipedia_dump(dump, is_subset=True)
    